In [1]:
from langchain.memory import ConversationSummaryBufferMemory 
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {"question": "Titanic", "answer": "🚢👩👨"},
    {"question": "The Devil Wears Prada", "answer": "😈👩🏻👵🏻"},
    {"question": "Elemental", "answer": "👨‍👩‍👧‍👦🔥💧"},
]

memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=120, 
    return_messages=True
)

example_prompt = ChatPromptTemplate.from_messages([ 
    ("human", "{question}"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie critic.If you were asked to answer about movie, you must closely follow the examples given and answer with three emojis that clearly represents the movie being asked about. Else, please respond kindly in sentence."),
    MessagesPlaceholder(variable_name="history"), 
    example_prompt,
    ("human", "{question}"),
])

# 메모리 변수 획득 함수
def load_memory(_):
    return memory.load_memory_variables({})["history"] 

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):

    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

invoke_chain("zootopia")

🐰🦊🐾content='🐰🦊🐾'


In [2]:
invoke_chain("ailien")

👽👾🚀content='👽👾🚀'


In [3]:
invoke_chain("Which movie was it I first asked?")

Zootopiacontent='Zootopia'
